In [3]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import pandas as pd

In [4]:
sentiment_analyzer = pipeline("sentiment-analysis", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", top_k=None)

# Test the model with a sentence
result = sentiment_analyzer("improves GDP by 5% and increases homelessness by 5%")
print(result)

[[{'label': 'positive', 'score': 0.7101832628250122}, {'label': 'negative', 'score': 0.17953164875507355}, {'label': 'neutral', 'score': 0.11028500646352768}]]


In [5]:
model_name ="lxyuan/distilbert-base-multilingual-cased-sentiments-student"

In [6]:
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name)

## Final data

In [7]:
df_train = pd.read_csv("train_harris_sentiment.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train_harris_sentiment.csv'

In [ ]:
df_train.head()

In [ ]:
# Drop the 'candidate' and 'publishedAt' columns
df_train_final = df_train.drop(columns=['candidate', 'publishedAt'])

df_train_final.head()

# Save the modified dataset (Once)

In [ ]:
#df_train_final.to_csv("harris_train_final.csv", index=False)

# Tokens

In [8]:
# check if all words in articles have tokens

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Print out words not found in tokenizer
def check_tokens_in_vocab(text, tokenizer):
    words = text.split()  # Split the article into words
    missing_tokens = []
    for word in words:
        tokenized_word = tokenizer.tokenize(word)
        if len(tokenized_word) == 0:  # If the tokenizer returns an empty list, the word is not tokenized
            print(f"Word not in tokenizer: {word}")
            missing_tokens.append(word)
    return missing_tokens

# Loop through all articles in the dataframe
df_train_final['missing_tokens'] = df_train_final['article'].apply(lambda x: check_tokens_in_vocab(x, tokenizer))

# Check the missing tokens for each article
missing_tokens = df_train_final['missing_tokens'].explode().dropna().unique()
print(f"Unique missing tokens: {missing_tokens}")

# Optionally, display the missing tokens for review
print(f"Total unique missing tokens: {len(missing_tokens)}")

NameError: name 'df_train_final' is not defined

# Check if some important words from word cloud are tokens if not add 

In [12]:
words = ['tim walz', 'lie', 'abortion', 'news', 'post', 'debate', 
        'vp', 'cnn', 'rally', 'attack', 'told', 'vice president', 'campaign',
         'appeared', 'voter', 'economy', 'rant', 
         'republican', 'pennsylvania', 'democratic', 'vote', 'speech', 'right', 'georgia', 'support', 
         'policy', 'liz cheney', 'endorsed', 'gop', 'endorsement', 'country', 'video', 
         'abc', 'host', 'among', 'joe', 'democrat', 'former', 'try', 'claimed', 'assassination', 
         'political', 'people', 'administration', 'fox',
        'voting', 'white', 'win', 'american', 'health', 'biden',
        'record', 'poll', 'house', 'woman', 'lead', 'candidate', 
         'maga', 'state', 'wisconsin', 'election', 'black', 
         'nominee', 'report', 'year', 'world', 'live', 'endorses', 
         'presidential', 'plan', 'said', 'america', 'would', 'back', 'supporter', 'border', 'view', 'charlamagne', 
         'hurricane', 'podcast', 'interview', 'msnbc', 'elon musk', 'medical', 'town hall', 
         'bret baier', 'protrump', 'carolina']



# Function to check if each word in the array is tokenized
def check_multiple_words_tokenized(words_list, tokenizer):
    for word in words_list:
        tokenized_word = tokenizer.tokenize(word)
        print(f"Tokenized version of '{word}': {tokenized_word}")
        
check_multiple_words_tokenized(words, tokenizer)

Tokenized version of 'tim walz': ['tim', 'wa', '##lz']
Tokenized version of 'lie': ['lie']
Tokenized version of 'abortion': ['ab', '##ortion']
Tokenized version of 'news': ['news']
Tokenized version of 'post': ['post']
Tokenized version of 'debate': ['debate']
Tokenized version of 'vp': ['v', '##p']
Tokenized version of 'cnn': ['cn', '##n']
Tokenized version of 'rally': ['rally']
Tokenized version of 'attack': ['attack']
Tokenized version of 'told': ['told']
Tokenized version of 'vice president': ['vice', 'president']
Tokenized version of 'campaign': ['campaign']
Tokenized version of 'appeared': ['appeared']
Tokenized version of 'voter': ['voter']
Tokenized version of 'economy': ['economy']
Tokenized version of 'rant': ['ran', '##t']
Tokenized version of 'republican': ['republic', '##an']
Tokenized version of 'pennsylvania': ['penn', '##sy', '##lva', '##nia']
Tokenized version of 'democratic': ['democratic']
Tokenized version of 'vote': ['vote']
Tokenized version of 'speech': ['speech'

In [13]:
# Check which words need to be added as tokens
missing_tokens = []
for word in words:
    tokenized_word = tokenizer.tokenize(word)
    if len(tokenized_word) > 1 or any(t.startswith("##") for t in tokenized_word):
        missing_tokens.append(word)

# Add missing tokens to the tokenizer
if missing_tokens:
    tokenizer.add_tokens(missing_tokens)
    print(f"Added {len(missing_tokens)} new tokens: {missing_tokens}")
else:
    print("No new tokens needed.")

# Resize the model’s token embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

Added 28 new tokens: ['tim walz', 'abortion', 'vp', 'cnn', 'vice president', 'rant', 'republican', 'pennsylvania', 'georgia', 'liz cheney', 'gop', 'endorsement', 'abc', 'joe', 'democrat', 'fox', 'biden', 'wisconsin', 'endorses', 'america', 'charlamagne', 'podcast', 'msnbc', 'elon musk', 'town hall', 'bret baier', 'protrump', 'carolina']


Embedding(119575, 768, padding_idx=0)

In [14]:
model.distilbert.embeddings

Embeddings(
  (word_embeddings): Embedding(119575, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

# Save pretrained model

In [15]:
model.save_pretrained("C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala")
tokenizer.save_pretrained("C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala")

('C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala\\tokenizer_config.json',
 'C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala\\special_tokens_map.json',
 'C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala\\vocab.txt',
 'C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala\\added_tokens.json',
 'C:/Users/User/Documents/3rd year/Datsci346/Project/DS346proj/model_with_tokens_kamala\\tokenizer.json')